About the Dataset:
1.id: unique id for a news article
2.title: the title of a news article
3.author:author of the news article
4.text:the text of the article;could be incomplete
5.label: a lable that marks whether the news article is real or fake;:

1: fake news
2: real news

Importing the dependencies

In [94]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.metrics import accuracy_score

In [51]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Gargi
[nltk_data]     Verma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [52]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-processing

In [53]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv('train.csv ')

In [54]:
news_dataset.shape

(20800, 5)

In [55]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [57]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [58]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+''+news_dataset['title']

In [59]:
print(news_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [60]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [61]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:
stemming is the process of reducing a word to its Root word
example: actor,actress,acting --> act

In [62]:
port_stem = PorterStemmer()

In [63]:
def stemming(content):
    stemmed_content = re.sub('[˄a-zA-Z],' ,' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content
                             

In [64]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [65]:
print(news_dataset['content'])

0        darrel lucushous dem aide: didn’t even see com...
1        daniel j. flynnflynn: hillari clinto big woman...
2               consortiumnews.comwhi truth might get fire
3        jessica purkiss15 civilian kill singl us airst...
4        howard portnoyiranian woman jail fiction unpub...
                               ...                        
20795    jerom hudsonrapp t.i.: trump ’poster child whi...
20796    benjamin hoffmann.f.l. playoffs: schedul match...
20797    michael j. de la merc rachel abramsmacy’ said ...
20798    alex ansarynat russia hold parallel exercis ba...
20799                     david swansonwhat keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [66]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [67]:
print(X)

['darrel lucushous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynnflynn: hillari clinto big woman campu - breitbart'
 'consortiumnews.comwhi truth might get fire' ...
 'michael j. de la merc rachel abramsmacy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansarynat russia hold parallel exercis balkan'
 'david swansonwhat keep f-35 aliv']


In [68]:
print (Y)

[1 0 1 ... 0 1 1]


In [69]:
Y.shape

(20800,)

In [70]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [71]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 205000 stored elements and shape (20800, 34091)>
  Coords	Values
  (0, 1101)	0.3604671545017939
  (0, 5207)	0.33776592683420303
  (0, 6107)	0.22834285721593095
  (0, 7271)	0.3266552014425343
  (0, 7619)	0.26385783954602704
  (0, 7925)	0.2723180261605273
  (0, 9703)	0.21276740389156013
  (0, 15182)	0.22661508799004057
  (0, 17122)	0.26928733132862315
  (0, 17646)	0.39585571595791763
  (0, 26487)	0.23614920498484515
  (0, 30619)	0.2678567402796535
  (1, 3401)	0.337367732677584
  (1, 4138)	0.17641776517844096
  (1, 4801)	0.437036977766507
  (1, 5838)	0.3832027806583411
  (1, 7211)	0.3010645351332899
  (1, 10616)	0.4973701984105556
  (1, 13465)	0.2387186288436683
  (1, 32498)	0.3495862435778659
  (2, 6257)	0.5492828216351007
  (2, 6414)	0.40403066024870815
  (2, 10384)	0.3475673273631621
  (2, 11412)	0.3063934315939872
  (2, 19131)	0.43149207118314675
  :	:
  (20797, 2982)	0.33040862541989496
  (20797, 7403)	0.212315582501208
  

splitting the dataset to training and test data

In [74]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,stratify = Y,random_state=2)

Training the model : Logistic Regression

In [75]:
model = LogisticRegression()

In [76]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation
accuracy score

In [77]:
# acccuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [78]:
print('Accuracy score of the training data :',training_data_accuracy)

Accuracy score of the training data : 0.9703125


In [79]:
# acccuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [80]:
print('Accuracy score of the test data :',test_data_accuracy)

Accuracy score of the test data : 0.9533653846153847


Making a predictive System

In [81]:
X_new = X_test[10]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
    print('The news is Real')
else:
    print('The news is Fake')

[0]
The news is Real


In [82]:
print(Y_test[10])

0


In [83]:
model = LinearRegression()

In [84]:
model.fit(X_train, Y_train)

LinearRegression()

In [85]:
# acccuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [92]:
print('Accuracy score of the training data :',training_data_accuracy)

Accuracy score of the training data : 0.9749399038461538


In [ ]:
# acccuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [86]:
print('Accuracy score of the test data :',test_data_accuracy)

Accuracy score of the test data : 0.9533653846153847


In [87]:
model = MultinomialNB()

In [88]:
model.fit(X_train,Y_train)

MultinomialNB()

In [90]:
# acccuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [91]:
print('Accuracy score of the training data :',training_data_accuracy)

Accuracy score of the training data : 0.9749399038461538


In [ ]:
# acccuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [93]:
print('Accuracy score of the test data :',test_data_accuracy)

Accuracy score of the test data : 0.9533653846153847


In [95]:
svm_model = LinearSVC()


In [96]:
model.fit(X_train,Y_train)

MultinomialNB()

In [97]:
# acccuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [98]:
print('Accuracy score of the training data :',training_data_accuracy)

Accuracy score of the training data : 0.9749399038461538


In [99]:
# acccuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [100]:
print('Accuracy score of the test data :',test_data_accuracy)

Accuracy score of the test data : 0.9257211538461538
